In [1]:
import os
from BH.feature_functions import *

GPU_NUM = "0"
num_epochs = 500
batch_size = 128

use_ppath = False
column_info = "original"

graph_deg = 5
num_layers = graph_deg
num_features = 256
feature_list = {
    'constant':    (False, constant_feature),
    'column':      (False, column_indicator),
    'norm_column': (True, normalized_column_indicator), #   Feauture
    'norm_column_rev': (False, normalized_column_rev_indicator),
}
connected = True
UPTO = False

direction = "222"

shape_indicator = {
    'all_with_all_row_connectedness_criterion': (False, ),
    'all_with_inductive_connectedness_criterion': (True, ),
    '2row_less': (False, ),
    '2row_less_with_all_row_connectedness_criterion': (False, ),
    '2row_less_with_inductive_connectedness_criterion': (False, ),
    '3row_less_with_all_row_connectedness_criterion': (False, ),
    '3row_less_with_inductive_connectedness_criterion': (False, ),
}

shape= {
    'all': (False, ),# 3개다 
    '2row_less': (True, ),
    '3row_less': (False, ),
}
filter_indicator = {
    'with_all_row_connectedness_criterion':(False,), 
    'with_inductive_connectedness_criterion':(False,),
}


use_pretrained_weights = False
save_trained_weights = True

step_size = 0.0001
train_fraction = .8

DIR_PATH = f'/Data/Ptab/n={graph_deg}'
MODEL_DIR = './trained_models'
MODEL_FILE = os.path.join(MODEL_DIR, f'parameters_{graph_deg}_{num_layers}_{num_features}')

for key in shape_indicator:
    if shape_indicator[key][0] == True:
        DIR_PATH += f'_{key}'
        MODEL_FILE += f'_{key}'


if connected == True:
    DIR_PATH += "_connected"
    MODEL_FILE += "_connected"
elif connected == False:
    DIR_PATH += "_disconnected"
    MODEL_FILE += "_disconnected"
if UPTO == True:
    DIR_PATH += "_UPTO"
    MODEL_FILE += "_UPTO"
if not column_info == "original":
    DIR_PATH += column_info
    MODEL_FILE += column_info
if use_ppath:
    DIR_PATH += "_ppath"    
    MODEL_FILE += "_ppath"
    
for key in feature_list.keys():
    if feature_list[key][0] == True:
        MODEL_FILE += f'_{key}'
MODEL_FILE += f'_{direction}'
MODEL_FILE += '.pickle'



<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
import functools
import enum
import os
from BH.data_loader import *
# from BH.generate_data import *
#from BH.data_loader import *
#from BH.generate_data import *
# from training_info import *

import pickle

from CustomDataset import *
from GCN_model import *

from torch_geometric.loader import DataLoader


os.environ["CUDA_VISIBLE_DEVICES"] = GPU_NUM
device = "cuda:" + GPU_NUM


print("Loading input data...")
full_dataset, train_dataset, test_dataset = load_input_data(DIR_PATH)

node_dim = num_features
edge_dim = 8
graph_deg = graph_deg
depth = num_layers

test_dataset = CustomDataset(test_dataset)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

train_dataset = CustomDataset(train_dataset)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = pastGCN().to(device)
if use_pretrained_weights == True:
    try:
        with open(MODEL_FILE, 'rb') as f:
          model, max_accuracy, min_loss = pickle.load(f)
          model.to(device)
    except:
        print("There is no trained model")
        use_pretrained_weights = False
if use_pretrained_weights == False:
    model = GCN_multi_conv(graph_deg, depth, node_dim, direction).to(device)
    max_accuracy = 0
    min_loss = 100

# data = batch.to(device)
# torch.nn.init.xavier_normal(model)
# loss_function = torch.nn.CrossEntropyLoss()
loss_function = torch.nn.NLLLoss()
# loss_function = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=step_size, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer=optimizer,
                                        lr_lambda=lambda epoch: 0.993 ** epoch,
                                        last_epoch=-1,
                                        verbose=False)

/root/anaconda3/envs/mpnn/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading input data...
Generating data from the directory /Data/Ptab/n=5_all_with_inductive_connectedness_criterion_connected


In [4]:
for epoch in range(num_epochs):
    # Training phase
    model.train()
    for batch in train_loader:
        batch.to(device)
        optimizer.zero_grad()
        out = model(batch)
        log_probs = torch.log(torch.clamp(out, min=1e-9))
#         log_probs = torch.log(out)
        loss = loss_function(log_probs, batch.y)
        loss.backward()
        # Apply gradient clipping
    
        optimizer.step()
    scheduler.step()
    print(loss)
    
    # Evaluation phase
    model.eval()
    correct = 0
    total = 0
#     with torch.no_grad():
#         for batch in test_loader:
#             batch.to(device)
#             outputs = model(batch)
# #             _,predicted = torch.max(outputs.data, 1)
#             predicted = outputs
#             total += batch.y.size(0)
# #             correct += (predicted == batch.y).sum().item()
#             correct += ((predicted - batch.y)**2<0.1).sum().item()
    with torch.no_grad():  # Disable gradient calculation
        total = 0
        correct = 0
        for batch in test_loader:
            # Move batch to the appropriate device (GPU or CPU)
            batch.to(device)

            # Forward pass: get model predictions
            outputs = model(batch)

            # Predicted classes are the indices of the max logits along the class dimension
            _, predicted = torch.max(outputs.data, 1)  # Shape: (batch_size)

            # Increment the total number of samples
            total += batch.y.size(0)

            # Compare predicted labels with ground truth labels
            correct += (predicted == batch.y).sum().item()  # Count the number of correct predictions

    # Compute the accuracy as a percentage
    accuracy = 100 * correct / total
    print(f'Accuracy: {accuracy:.2f}%')
    # Compute accuracy
    accuracy = correct / total
    loss = float(loss.item())

    print("Epoch [{}/{}], Accuracy: {:.2%}, Loss: {:.15f}".format(epoch + 1, num_epochs, accuracy, loss))

    if (accuracy > max_accuracy or (accuracy == max_accuracy and loss < min_loss)) and save_trained_weights:
        max_accuracy = accuracy
        min_loss = loss
        with open(MODEL_FILE, 'wb') as f:
            pickle.dump((model, max_accuracy, min_loss), f)

/root/Min/Ptab/CustomDataset.py:18: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484809662/work/torch/csrc/utils/tensor_new.cpp:201.)
  edge_index = torch.tensor([self.rows[idx], self.cols[idx]], dtype=torch.long)


tensor(1.2108, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 53.23%
Epoch [1/500], Accuracy: 53.23%, Loss: 1.210806250572205
tensor(1.1019, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 48.39%
Epoch [2/500], Accuracy: 48.39%, Loss: 1.101894974708557
tensor(0.6897, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 59.68%
Epoch [3/500], Accuracy: 59.68%, Loss: 0.689731121063232
tensor(0.9291, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 59.68%
Epoch [4/500], Accuracy: 59.68%, Loss: 0.929070353507996
tensor(0.5686, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 70.97%
Epoch [5/500], Accuracy: 70.97%, Loss: 0.568581283092499
tensor(0.6283, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 70.97%
Epoch [6/500], Accuracy: 70.97%, Loss: 0.628316462039948
tensor(0.5175, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 61.29%
Epoch [7/500], Accuracy: 61.29%, Loss: 0.517524063587189
tensor(0.5923, device='cuda:0', grad_fn=<NllLossBackward0>)
Ac

tensor(0.1393, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 88.71%
Epoch [62/500], Accuracy: 88.71%, Loss: 0.139343813061714
tensor(0.1115, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 93.55%
Epoch [63/500], Accuracy: 93.55%, Loss: 0.111529208719730
tensor(0.0992, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [64/500], Accuracy: 91.94%, Loss: 0.099163994193077
tensor(0.1372, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 93.55%
Epoch [65/500], Accuracy: 93.55%, Loss: 0.137191399931908
tensor(0.1627, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 93.55%
Epoch [66/500], Accuracy: 93.55%, Loss: 0.162669152021408
tensor(0.1474, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 88.71%
Epoch [67/500], Accuracy: 88.71%, Loss: 0.147392064332962
tensor(0.1104, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 93.55%
Epoch [68/500], Accuracy: 93.55%, Loss: 0.110428258776665
tensor(0.1381, device='cuda:0', grad_fn=<NllLossBackwar

tensor(0.0611, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [123/500], Accuracy: 91.94%, Loss: 0.061137270182371
tensor(0.0611, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [124/500], Accuracy: 91.94%, Loss: 0.061074346303940
tensor(0.0565, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 90.32%
Epoch [125/500], Accuracy: 90.32%, Loss: 0.056549936532974
tensor(0.0663, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [126/500], Accuracy: 91.94%, Loss: 0.066324763000011
tensor(0.0586, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [127/500], Accuracy: 91.94%, Loss: 0.058593325316906
tensor(0.0675, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [128/500], Accuracy: 91.94%, Loss: 0.067480504512787
tensor(0.0518, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [129/500], Accuracy: 91.94%, Loss: 0.051838040351868
tensor(0.0706, device='cuda:0', grad_fn=<NllLoss

tensor(0.0277, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [184/500], Accuracy: 91.94%, Loss: 0.027731394395232
tensor(0.0370, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [185/500], Accuracy: 91.94%, Loss: 0.037042167037725
tensor(0.0462, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [186/500], Accuracy: 91.94%, Loss: 0.046244483441114
tensor(0.0500, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [187/500], Accuracy: 91.94%, Loss: 0.049953415989876
tensor(0.0437, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [188/500], Accuracy: 91.94%, Loss: 0.043713916093111
tensor(0.0477, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [189/500], Accuracy: 91.94%, Loss: 0.047660902142525
tensor(0.0483, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [190/500], Accuracy: 91.94%, Loss: 0.048269342631102
tensor(0.0481, device='cuda:0', grad_fn=<NllLoss

tensor(0.0204, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 93.55%
Epoch [245/500], Accuracy: 93.55%, Loss: 0.020354256033897
tensor(0.0265, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [246/500], Accuracy: 91.94%, Loss: 0.026549460366368
tensor(0.0204, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [247/500], Accuracy: 91.94%, Loss: 0.020417969673872
tensor(0.0340, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 93.55%
Epoch [248/500], Accuracy: 93.55%, Loss: 0.034035071730614
tensor(0.0248, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [249/500], Accuracy: 91.94%, Loss: 0.024823769927025
tensor(0.0221, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [250/500], Accuracy: 91.94%, Loss: 0.022101489827037
tensor(0.0288, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 93.55%
Epoch [251/500], Accuracy: 93.55%, Loss: 0.028823181986809
tensor(0.0277, device='cuda:0', grad_fn=<NllLoss

tensor(0.0212, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [306/500], Accuracy: 91.94%, Loss: 0.021235072985291
tensor(0.0137, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [307/500], Accuracy: 91.94%, Loss: 0.013728990219533
tensor(0.0302, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 93.55%
Epoch [308/500], Accuracy: 93.55%, Loss: 0.030197162181139
tensor(0.0244, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [309/500], Accuracy: 91.94%, Loss: 0.024417201057076
tensor(0.0184, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [310/500], Accuracy: 91.94%, Loss: 0.018354794010520
tensor(0.0152, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [311/500], Accuracy: 91.94%, Loss: 0.015227253548801
tensor(0.0284, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [312/500], Accuracy: 91.94%, Loss: 0.028365986421704
tensor(0.0190, device='cuda:0', grad_fn=<NllLoss

tensor(0.0224, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [368/500], Accuracy: 91.94%, Loss: 0.022411560639739
tensor(0.0202, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [369/500], Accuracy: 91.94%, Loss: 0.020224239677191
tensor(0.0174, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [370/500], Accuracy: 91.94%, Loss: 0.017352800816298
tensor(0.0236, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [371/500], Accuracy: 91.94%, Loss: 0.023636948317289
tensor(0.0227, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [372/500], Accuracy: 91.94%, Loss: 0.022686896845698
tensor(0.0133, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [373/500], Accuracy: 91.94%, Loss: 0.013300610706210
tensor(0.0175, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [374/500], Accuracy: 91.94%, Loss: 0.017468530684710
tensor(0.0168, device='cuda:0', grad_fn=<NllLoss

tensor(0.0183, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [430/500], Accuracy: 91.94%, Loss: 0.018325289711356
tensor(0.0136, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [431/500], Accuracy: 91.94%, Loss: 0.013592174276710
tensor(0.0170, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [432/500], Accuracy: 91.94%, Loss: 0.017003728076816
tensor(0.0194, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [433/500], Accuracy: 91.94%, Loss: 0.019406853243709
tensor(0.0226, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [434/500], Accuracy: 91.94%, Loss: 0.022649561986327
tensor(0.0216, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [435/500], Accuracy: 91.94%, Loss: 0.021646630018950
tensor(0.0205, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [436/500], Accuracy: 91.94%, Loss: 0.020502628758550
tensor(0.0160, device='cuda:0', grad_fn=<NllLoss

tensor(0.0153, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [492/500], Accuracy: 91.94%, Loss: 0.015320833772421
tensor(0.0217, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [493/500], Accuracy: 91.94%, Loss: 0.021713426336646
tensor(0.0172, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [494/500], Accuracy: 91.94%, Loss: 0.017249194905162
tensor(0.0156, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [495/500], Accuracy: 91.94%, Loss: 0.015551581047475
tensor(0.0153, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [496/500], Accuracy: 91.94%, Loss: 0.015314643271267
tensor(0.0144, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [497/500], Accuracy: 91.94%, Loss: 0.014436551369727
tensor(0.0165, device='cuda:0', grad_fn=<NllLossBackward0>)
Accuracy: 91.94%
Epoch [498/500], Accuracy: 91.94%, Loss: 0.016457609832287
tensor(0.0144, device='cuda:0', grad_fn=<NllLoss

In [50]:
for batch in train_loader:
    print(batch)
    batch.to(device)
    out = model(batch)
    out_single = model.GCN_single(batch)
    break

DataBatch(x=[695, 1], edge_index=[2, 2085], y=[128], edge_types=[2085, 1], batch=[695], ptr=[129])


In [55]:
batch.batch

tensor([  0,   0,   0,   0,   0,   1,   1,   1,   1,   1,   2,   2,   2,   2,
          2,   3,   3,   3,   3,   3,   4,   4,   4,   4,   4,   5,   5,   5,
          5,   5,   6,   6,   6,   6,   6,   7,   7,   7,   7,   7,   8,   8,
          8,   8,   8,   9,   9,   9,   9,   9,  10,  10,  10,  10,  10,  11,
         11,  11,  11,  11,  11,  11,  11,  11,  11,  11,  11,  11,  11,  11,
         12,  12,  12,  12,  12,  13,  13,  13,  13,  13,  14,  14,  14,  14,
         14,  15,  15,  15,  15,  15,  16,  16,  16,  16,  16,  17,  17,  17,
         17,  17,  18,  18,  18,  18,  18,  19,  19,  19,  19,  19,  20,  20,
         20,  20,  20,  21,  21,  21,  21,  21,  22,  22,  22,  22,  22,  23,
         23,  23,  23,  23,  24,  24,  24,  24,  24,  25,  25,  25,  25,  25,
         26,  26,  26,  26,  26,  27,  27,  27,  27,  27,  28,  28,  28,  28,
         28,  29,  29,  29,  29,  29,  30,  30,  30,  30,  30,  31,  31,  31,
         31,  31,  32,  32,  32,  32,  32,  33,  33,  33,  33,  

In [49]:
out_single.shape

torch.Size([140, 1])

In [36]:
out.shape

torch.Size([128, 4])

In [17]:
torch.reshape(batch.x,(-1,5,7))

tensor([[[1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1.]],

        [[1., 1

In [14]:
batch.batch

tensor([  0,   0,   0,   0,   0,   1,   1,   1,   1,   1,   2,   2,   2,   2,
          2,   3,   3,   3,   3,   3,   4,   4,   4,   4,   4,   5,   5,   5,
          5,   5,   6,   6,   6,   6,   6,   7,   7,   7,   7,   7,   8,   8,
          8,   8,   8,   9,   9,   9,   9,   9,  10,  10,  10,  10,  10,  11,
         11,  11,  11,  11,  12,  12,  12,  12,  12,  13,  13,  13,  13,  13,
         14,  14,  14,  14,  14,  15,  15,  15,  15,  15,  16,  16,  16,  16,
         16,  17,  17,  17,  17,  17,  18,  18,  18,  18,  18,  19,  19,  19,
         19,  19,  20,  20,  20,  20,  20,  21,  21,  21,  21,  21,  22,  22,
         22,  22,  22,  23,  23,  23,  23,  23,  24,  24,  24,  24,  24,  25,
         25,  25,  25,  25,  26,  26,  26,  26,  26,  27,  27,  27,  27,  27,
         28,  28,  28,  28,  28,  28,  28,  28,  28,  28,  29,  29,  29,  29,
         29,  30,  30,  30,  30,  30,  31,  31,  31,  31,  31,  32,  32,  32,
         32,  32,  33,  33,  33,  33,  33,  34,  34,  34,  34,  

In [11]:
(batch.edge_types==3).squeeze()

tensor([False, False, False,  ..., False, False, False], device='cuda:0')

In [12]:
batch.edge_index[:,(batch.edge_types==3).squeeze()]

tensor([[  0,   1,   2,   5,   5,   6,   6,   7,  10,  10,  10,  11,  15,  15,
          15,  16,  16,  17,  20,  20,  20,  21,  21,  22,  25,  25,  25,  26,
          26,  30,  30,  31,  35,  35,  40,  40,  45,  46,  46,  47,  50,  50,
          51,  55,  55,  55,  56,  61,  62,  65,  65,  66,  67,  70,  70,  71,
          75,  75,  76,  80,  80,  81,  85,  85,  85,  86,  87,  90,  90,  91,
          95,  95,  96,  97, 100, 100, 101, 101, 102, 105, 105, 106, 107, 110,
         110, 111, 115, 115, 116, 120, 121, 121, 125, 125, 126, 130, 130, 130,
         131, 135, 135, 136, 137, 140, 141, 145, 146, 150, 150, 155, 156, 156,
         157, 160, 160, 160, 161, 161, 165, 165, 166, 170, 170, 170, 171, 171,
         172, 175, 175, 175, 176, 176, 177, 181, 185, 185, 186, 190, 190, 191,
         192, 195, 196, 196, 201, 205, 205, 206, 210, 210, 212, 215, 215, 216,
         217, 220, 220, 221, 225, 225, 225, 226, 227, 230, 230, 235, 235, 241,
         242, 245, 250, 255, 255, 255, 256, 257, 260

In [23]:
row_counts = torch.bincount(batch.batch, minlength=batch_size)
max_columns = row_counts.max().item()
col_indices = torch.zeros_like(batch.batch)

In [25]:
for i in range(batch_size):
    col_indices[batch.batch == i] = torch.arange(row_counts[i],device=device)


In [26]:
col_indices

tensor([ 0,  1,  2,  3,  4,  0,  1,  2,  3,  4,  0,  1,  2,  3,  4,  0,  1,  2,
         3,  4,  0,  1,  2,  3,  4,  0,  1,  2,  3,  4,  0,  1,  2,  3,  4,  0,
         1,  2,  3,  4,  0,  1,  2,  3,  4,  0,  1,  2,  3,  4,  0,  1,  2,  3,
         4,  0,  1,  2,  3,  4,  0,  1,  2,  3,  4,  0,  1,  2,  3,  4,  0,  1,
         2,  3,  4,  0,  1,  2,  3,  4,  0,  1,  2,  3,  4,  0,  1,  2,  3,  4,
         0,  1,  2,  3,  4,  0,  1,  2,  3,  4,  0,  1,  2,  3,  4,  0,  1,  2,
         3,  4,  0,  1,  2,  3,  4,  0,  1,  2,  3,  4,  0,  1,  2,  3,  4,  0,
         1,  2,  3,  4,  0,  1,  2,  3,  4,  0,  1,  2,  3,  4,  0,  1,  2,  3,
         4,  5,  6,  7,  8,  9,  0,  1,  2,  3,  4,  0,  1,  2,  3,  4,  0,  1,
         2,  3,  4,  0,  1,  2,  3,  4,  0,  1,  2,  3,  4,  0,  1,  2,  3,  4,
         0,  1,  2,  3,  4,  0,  1,  2,  3,  4,  0,  1,  2,  3,  4,  0,  1,  2,
         3,  4,  0,  1,  2,  3,  4,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9,  0,
         1,  2,  3,  4,  0,  1,  2,  3, 

In [40]:
node_dim = 1

In [41]:
xx=torch.reshape(batch.x,(-1,node_dim))

In [45]:
zero_placeholder =  torch.full((batch_size, max_columns,node_dim), 0,device=device)

In [44]:
zero_placeholder[batch.batch]

tensor([[[-inf],
         [-inf],
         [-inf],
         ...,
         [-inf],
         [-inf],
         [-inf]],

        [[-inf],
         [-inf],
         [-inf],
         ...,
         [-inf],
         [-inf],
         [-inf]],

        [[-inf],
         [-inf],
         [-inf],
         ...,
         [-inf],
         [-inf],
         [-inf]],

        ...,

        [[-inf],
         [-inf],
         [-inf],
         ...,
         [-inf],
         [-inf],
         [-inf]],

        [[-inf],
         [-inf],
         [-inf],
         ...,
         [-inf],
         [-inf],
         [-inf]],

        [[-inf],
         [-inf],
         [-inf],
         ...,
         [-inf],
         [-inf],
         [-inf]]], device='cuda:0')

In [33]:
batch.batch.shape

torch.Size([700])

In [ ]:
loss = loss_function(torch.log(out), batch.y)